# Overview

We use reviews to predict the rating of Amazon Instant Video by CNN, GRU and LSTM model. Besides, we use NLP to manipulate reviews.

# Load the packages

In [1]:
import pandas as pd
import numpy as np
import json
from time import time
from sklearn.model_selection import train_test_split

In [2]:
def get_list_of_dicts(fname):
    return [json.loads(i) for i in open(fname,'rt')]

raw_data = get_list_of_dicts('Amazon_Instant_Video_5.json')

data = pd.DataFrame(raw_data).loc[:, ["reviewerID",'reviewText', "asin", "overall"]]
data.head(10)   

,reviewerID,reviewText,asin,overall
0,A11N155CW1UV02,I had big expectations because I love English ...,B000H00VBQ,2.0
1,A3BC8O2KCL29V2,I highly recommend this series. It is a must f...,B000H00VBQ,5.0
2,A60D5HQFOTSOM,This one is a real snoozer. Don't believe anyt...,B000H00VBQ,1.0
3,A1RJPIGRSNX4PW,Mysteries are interesting. The tension betwee...,B000H00VBQ,4.0
4,A16XRPF40679KG,"This show always is excellent, as far as briti...",B000H00VBQ,5.0
5,A1POFVVXUZR3IQ,I discovered this series quite by accident. Ha...,B000H00VBQ,5.0
6,A1PG2VV4W1WRPL,"It beats watching a blank screen. However, I j...",B000H0X79O,3.0
7,ATASGS8HZHGIB,"There are many episodes in this series, so I p...",B000H0X79O,3.0
8,A3RXD7Z44T9DHW,This is the best of the best comedy Stand-up. ...,B000H0X79O,5.0
9,AUX8EUBNTHIIU,Not bad. Didn't know any of the comedians but...,B000H0X79O,3.0


# Data preprocessing

In [3]:
def add_user_reviews(x):
    ur = user_reviews.loc[x['reviewerID']].drop(x['asin']).values.tolist()
    mr = movie_reviews.loc[x['asin']].drop(x['reviewerID']).values.tolist()
    x['userReviews'] = ' '.join(list(map(lambda x: x[0], ur)))
    x['movieReviews'] = ' '.join(list(map(lambda x: x[0], mr)))
    return x

user_item_review = data.drop('reviewText', axis = 1)
user_reviews = pd.pivot_table(data, index = ['reviewerID','asin'], aggfunc = lambda x: x).drop('overall', axis = 1)
movie_reviews = pd.pivot_table(data, index = ['asin', 'reviewerID'], aggfunc = lambda x: x).drop('overall', axis = 1)

df = user_item_review.apply(add_user_reviews, axis = 1)
df.head(10)

,reviewerID,asin,overall,userReviews,movieReviews
0,A11N155CW1UV02,B000H00VBQ,2.0,I really like the characters and the actors. I...,"This show always is excellent, as far as briti..."
1,A3BC8O2KCL29V2,B000H00VBQ,5.0,This is one good show. It is interesting in it...,I had big expectations because I love English ...
2,A60D5HQFOTSOM,B000H00VBQ,1.0,I watched this a couple of weeks ago. There ar...,I had big expectations because I love English ...
3,A1RJPIGRSNX4PW,B000H00VBQ,4.0,"The acting was excellent. The acting, the rel...",I had big expectations because I love English ...
4,A16XRPF40679KG,B000H00VBQ,5.0,As many people said this show kept getting bet...,I had big expectations because I love English ...
5,A1POFVVXUZR3IQ,B000H00VBQ,5.0,Auschwitz: Inside the Nazi State makes for com...,I had big expectations because I love English ...
6,A1PG2VV4W1WRPL,B000H0X79O,3.0,Very slow moving plot if there is one. Trying...,This is the best of the best comedy Stand-up. ...
7,ATASGS8HZHGIB,B000H0X79O,3.0,"I think there are 10 episodes in this series, ...","It beats watching a blank screen. However, I j..."
8,A3RXD7Z44T9DHW,B000H0X79O,5.0,All the comedians are hilarious. I have seen t...,"It beats watching a blank screen. However, I j..."
9,AUX8EUBNTHIIU,B000H0X79O,3.0,"Yes, and who cares. The imagination can conju...","It beats watching a blank screen. However, I j..."


In [4]:
# train test split 
test_size = 0.005

# get test size percentage of users
unique_users = df.loc[:, 'reviewerID'].unique()
users_size = len(unique_users)
test_idx = np.random.choice(users_size, size = int(users_size * test_size), replace = False)

# get test users
test_users = unique_users[test_idx]

# everyone else is a training user
train_users = np.delete(unique_users, test_idx)

test = df[df['reviewerID'].isin(test_users)]
train = df[df['reviewerID'].isin(train_users)]

unique_test_movies = test['asin'].unique()
# drop the movies that also appear in our test set. in order to be
# a true train/test split, we are forced to discard some data entirely
train = train.where(np.logical_not(train['asin'].isin(unique_test_movies))).dropna()

train.head()

,reviewerID,asin,overall,userReviews,movieReviews
0,A11N155CW1UV02,B000H00VBQ,2.0,I really like the characters and the actors. I...,"This show always is excellent, as far as briti..."
1,A3BC8O2KCL29V2,B000H00VBQ,5.0,This is one good show. It is interesting in it...,I had big expectations because I love English ...
2,A60D5HQFOTSOM,B000H00VBQ,1.0,I watched this a couple of weeks ago. There ar...,I had big expectations because I love English ...
3,A1RJPIGRSNX4PW,B000H00VBQ,4.0,"The acting was excellent. The acting, the rel...",I had big expectations because I love English ...
4,A16XRPF40679KG,B000H00VBQ,5.0,As many people said this show kept getting bet...,I had big expectations because I love English ...


Embed the reviews into glove word2vect model

The pre-trained model is downloadable at https://nlp.stanford.edu/projects/glove/

In [5]:
import os
# functions to embed user reviews into the glove word2vect model
def init_embeddings_map(fname):
    with open(os.path.join("glove.6B", fname), encoding = 'utf8') as glove:
        return {l[0]: np.asarray(l[1:], dtype = 'float32') for l in [line.split() for line in glove]}

def get_embed_func(i_len, u_len, pad_value, embedding_map):
    def embed(row):
        sentence = row['userReviews'].split()[:u_len]
        reviews = list(map(lambda word: embedding_map.get(word) if word in embedding_map else pad_value, sentence))
        row['userReviews'] = reviews + [pad_value] * (u_len - len(reviews))
        sentence = row['movieReviews'].split()[:i_len]
        reviews = list(map(lambda word: embedding_map.get(word) if word in embedding_map else pad_value, sentence))
        row['movieReviews'] = reviews + [pad_value] * (i_len - len(reviews))
        return row
    return embed

In [6]:
emb_size  = 50
embedding_map = init_embeddings_map('glove.6B.' + str(emb_size) + 'd.txt')

user_sizes = df.loc[:, 'userReviews'].apply(lambda x: x.split()).apply(len)
item_sizes = df.loc[:, 'movieReviews'].apply(lambda x: x.split()).apply(len)

u_ptile = 40
i_ptile = 15
u_len = int(np.percentile(user_sizes, u_ptile))
i_len = int(np.percentile(item_sizes, i_ptile))

embedding_fn = get_embed_func(i_len, u_len, np.array([0.0] * emb_size), embedding_map)

train_embedded = train.apply(embedding_fn, axis = 1)
test_embedded = test.apply(embedding_fn, axis = 1)

print(u_len, i_len) # size of input in deep neural networks, useful to set parameter
train_embedded.head()

241 722


,reviewerID,asin,overall,userReviews,movieReviews
0,A11N155CW1UV02,B000H00VBQ,2.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,A3BC8O2KCL29V2,B000H00VBQ,5.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,A60D5HQFOTSOM,B000H00VBQ,1.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,A1RJPIGRSNX4PW,B000H00VBQ,4.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,A16XRPF40679KG,B000H00VBQ,5.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


# Deep learning models

In [7]:
import tensorflow as tf
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from tensorflow.python.keras.layers import Input, Dense, Dropout
from tensorflow.python.keras.layers.merge import Add, Dot, Concatenate
from tensorflow.python.keras.layers import Conv1D ,GRU, LSTM, MaxPooling1D, Flatten

## CNN model

In [8]:
def cnn_tower(max_len, embedding_size, hidden_size, filters=4, kernel_size=10):
        input_layer = Input(shape=(max_len, embedding_size))
        tower = Conv1D(filters=filters, kernel_size=kernel_size, activation="tanh")(input_layer)
        tower = MaxPooling1D()(tower)
        tower = Conv1D(filters=filters, kernel_size=kernel_size, activation="tanh")(tower)
        tower = MaxPooling1D()(tower)
        tower = Flatten()(tower)
        tower = Dense(hidden_size, activation="relu")(tower)
        tower = Dropout(0.4)(tower)
        return input_layer, tower
    
def CNN_model(embedding_size, hidden_size, u_len, i_len):
    inputU, towerU = cnn_tower(u_len, embedding_size, hidden_size)
    inputM, towerM = cnn_tower(i_len, embedding_size, hidden_size)
    joined = Concatenate()([towerU, towerM])
    outNeuron = Dense(1)(joined)
    dotproduct = Dot(axes=1)([towerU, towerM])
    output_layer = Add()([outNeuron, dotproduct])
        
    model = Model(inputs=[inputU, inputM], outputs=[output_layer])
    return model

hidden_size = 64

model_cnn = CNN_model(emb_size, hidden_size, u_len, i_len)
model_cnn.compile(optimizer='Adam', loss='mse')
model_cnn.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 241, 50)]    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 722, 50)]    0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 232, 4)       2004        input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 713, 4)       2004        input_2[0][0]                    
______________________________________________________________________________________________

In [9]:
batch_size = 32
epochs = 20

user_reviews = np.array(list(train_embedded.loc[:, "userReviews"]))
movie_reviews = np.array(list(train_embedded.loc[:, "movieReviews"]))

train_inputs = [user_reviews, movie_reviews]
train_outputs = train_embedded.loc[:, "overall"]

tensorboard = TensorBoard(log_dir="cnn_log")
earlystop = EarlyStopping(monitor='val_loss', patience=3)
checkpoint = ModelCheckpoint('cnn_weights.{epoch:02d}-{val_loss:.2f}.h5', monitor='val_loss', save_best_only=True)
train_history = model_cnn.fit(train_inputs, train_outputs, callbacks=[tensorboard, earlystop, checkpoint], 
                              validation_split=0.05, batch_size=batch_size, epochs=epochs)

model_cnn.save("cnn.h5")

C:\Users\ching\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 1/20
759/759 [==============================] - 23s 28ms/step - loss: 1.7714 - val_loss: 1.7490
Epoch 2/20
759/759 [==============================] - 19s 25ms/step - loss: 1.4364 - val_loss: 1.8209
Epoch 3/20
759/759 [==============================] - 20s 26ms/step - loss: 1.3277 - val_loss: 1.6010
Epoch 4/20
759/759 [==============================] - 20s 26ms/step - loss: 1.2964 - val_loss: 1.5319
Epoch 5/20
759/759 [==============================] - 19s 25ms/step - loss: 1.2137 - val_loss: 1.5043
Epoch 6/20
759/759 [==============================] - 19s 25ms/step - loss: 1.1926 - val_loss: 1.5152
Epoch 7/20
759/759 [==============================] - 18s 24ms/step - loss: 1.1624 - val_loss: 1.5821
Epoch 8/20
759/759 [==============================] - 18s 24ms/step - loss: 1.1535 - val_loss: 1.5351


## GRU model

In [10]:
def gru_tower(max_len, embedding_size, hidden_size, rnn_hidden_size, filters=2, kernel_size=8):
        input_layer = Input(shape=(max_len, embedding_size))
        tower = GRU(rnn_hidden_size, activation="tanh")(input_layer)
        tower = Dense(hidden_size, activation="relu")(tower)
        tower = Dropout(0.4)(tower)
        return input_layer, tower
    
def GRU_model(embedding_size, hidden_size, rnn_hidden_size, u_len, i_len):
    inputU, towerU = gru_tower(u_len, embedding_size, hidden_size, rnn_hidden_size)
    inputM, towerM = gru_tower(i_len, embedding_size, hidden_size, rnn_hidden_size)
    joined = Concatenate()([towerU, towerM])
    outNeuron = Dense(1)(joined)
    dotproduct = Dot(axes=1)([towerU, towerM])
    output_layer = Add()([outNeuron, dotproduct])
        
    model = Model(inputs=[inputU, inputM], outputs=[output_layer])
    return model

hidden_size = 64
rnn_hidden_size = 64

model_gru = GRU_model(emb_size, hidden_size, rnn_hidden_size, u_len, i_len)
model_gru.compile(optimizer='Adam', loss='mse')
model_gru.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 241, 50)]    0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 722, 50)]    0                                            
__________________________________________________________________________________________________
gru (GRU)                       (None, 64)           22080       input_3[0][0]                    
__________________________________________________________________________________________________
gru_1 (GRU)                     (None, 64)           22080       input_4[0][0]                    
____________________________________________________________________________________________

In [11]:
batch_size = 32
epochs = 20

tensorboard = TensorBoard(log_dir="gru_log")
earlystop = EarlyStopping(monitor='val_loss', patience=3)
checkpoint = ModelCheckpoint('gru_weights.{epoch:02d}-{val_loss:.2f}.h5', monitor='val_loss', save_best_only=True)
train_history = model_gru.fit(train_inputs, train_outputs, callbacks=[tensorboard, earlystop, checkpoint], 
                              validation_split=0.05, batch_size=batch_size, epochs=epochs)

model_gru.save("gru.h5")

Epoch 1/20
759/759 [==============================] - 385s 502ms/step - loss: 1.9333 - val_loss: 1.8281
Epoch 2/20
759/759 [==============================] - 409s 539ms/step - loss: 1.3990 - val_loss: 1.5182
Epoch 3/20
759/759 [==============================] - 445s 586ms/step - loss: 1.3334 - val_loss: 1.4807
Epoch 4/20
759/759 [==============================] - 513s 676ms/step - loss: 1.2762 - val_loss: 1.4487
Epoch 5/20
759/759 [==============================] - 502s 662ms/step - loss: 1.2135 - val_loss: 1.4375
Epoch 6/20
759/759 [==============================] - 509s 670ms/step - loss: 1.2208 - val_loss: 1.5109
Epoch 7/20
759/759 [==============================] - 499s 657ms/step - loss: 1.1702 - val_loss: 1.4504
Epoch 8/20
759/759 [==============================] - 511s 674ms/step - loss: 1.1139 - val_loss: 1.4199
Epoch 9/20
759/759 [==============================] - 525s 692ms/step - loss: 1.0737 - val_loss: 1.4260
Epoch 10/20
759/759 [==============================] - 558s 735m

## LSTM model

In [12]:
# LSTM model
def lstm_tower(max_len, embedding_size, hidden_size, rnn_hidden_size, filters=2, kernel_size=8):
        input_layer = Input(shape=(max_len, embedding_size))
        tower = LSTM(rnn_hidden_size, activation="tanh")(input_layer)
        tower = Dense(hidden_size, activation="relu")(tower)
        tower = Dropout(0.4)(tower)
        return input_layer, tower
    
def LSTM_model(embedding_size, hidden_size, rnn_hidden_size, u_len, i_len):
    inputU, towerU = lstm_tower(u_len, embedding_size, hidden_size, rnn_hidden_size)
    inputM, towerM = lstm_tower(i_len, embedding_size, hidden_size, rnn_hidden_size)
    joined = Concatenate()([towerU, towerM])
    outNeuron = Dense(1)(joined)
    dotproduct = Dot(axes=1)([towerU, towerM])
    output_layer = Add()([outNeuron, dotproduct])
        
    model = Model(inputs=[inputU, inputM], outputs=[output_layer])
    return model


hidden_size = 64
rnn_hidden_size = 64

model_lstm = LSTM_model(emb_size, hidden_size, rnn_hidden_size, u_len, i_len)
model_lstm.compile(optimizer='Adam', loss='mse')
model_lstm.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 241, 50)]    0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 722, 50)]    0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 64)           29440       input_5[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 64)           29440       input_6[0][0]                    
____________________________________________________________________________________________

In [16]:
batch_size = 32
epochs = 30

tensorboard = TensorBoard(log_dir="lstm_log")
earlystop = EarlyStopping(monitor='val_loss', patience=3)
checkpoint = ModelCheckpoint('lstm_weights.{epoch:02d}-{val_loss:.2f}.h5', monitor='val_loss', save_best_only=True)
train_history = model_lstm.fit(train_inputs, train_outputs, callbacks=[tensorboard, earlystop, checkpoint], 
                              validation_split=0.05, batch_size=batch_size, epochs=epochs)

model_lstm.save("lstm.h5")

Epoch 1/30
759/759 [==============================] - 974s 1s/step - loss: 0.9659 - val_loss: 1.4822
Epoch 2/30
759/759 [==============================] - 1000s 1s/step - loss: 0.9342 - val_loss: 1.4918
Epoch 3/30
759/759 [==============================] - 1035s 1s/step - loss: 0.9191 - val_loss: 1.4908
Epoch 4/30
759/759 [==============================] - 1037s 1s/step - loss: 0.8906 - val_loss: 1.4743
Epoch 5/30
759/759 [==============================] - 1051s 1s/step - loss: 0.8608 - val_loss: 1.5421
Epoch 6/30
759/759 [==============================] - 1056s 1s/step - loss: 0.8446 - val_loss: 1.4880
Epoch 7/30
759/759 [==============================] - 1069s 1s/step - loss: 0.8161 - val_loss: 1.4727
Epoch 8/30
759/759 [==============================] - 1081s 1s/step - loss: 0.7907 - val_loss: 1.4770
Epoch 9/30
759/759 [==============================] - 1099s 1s/step - loss: 0.7702 - val_loss: 1.5020
Epoch 10/30
759/759 [==============================] - 1131s 1s/step - loss: 0.7541

In [17]:
user_reviews = np.array(list(test_embedded.loc[:, "userReviews"]))
movie_reviews = np.array(list(test_embedded.loc[:, "movieReviews"]))
test_inputs = [user_reviews, movie_reviews]

true_rating = np.array(list(test_embedded.loc[:, "overall"])).reshape((-1, 1))

predictions_cnn = model_cnn.predict(test_inputs)
predictions_gru = model_gru.predict(test_inputs)
predictions_lstm = model_lstm.predict(test_inputs)

error_cnn = np.square(predictions_cnn - true_rating)
print("Test MSE for CNN model :", np.average(error_cnn))

error_gru = np.square(predictions_gru - true_rating)
print("Test MSE for GRU model :", np.average(error_gru))

error_lstm = np.square(predictions_lstm - true_rating)
print("Test MSE for LSTM model :", np.average(error_lstm))

Test MSE for CNN model : 1.5703979231145906
Test MSE for GRU model : 1.4958523094066452
Test MSE for LSTM model : 1.484589167566555


# Conclusion

The data is sparse. We need to run more epochs or collect more data.

1. RNN is better than CNN, because the reviews are sequential data.

2. LSTM is better than GRU.